<a href="https://colab.research.google.com/github/HausReport/MightyLogic/blob/master/Notebooks/Turf_War_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up environment


In [1]:

import sys
# need newer plotly/plotly_express than collab provides?
#!{sys.executable} -m pip install --force-reinstall --upgrade "plotly"
#!{sys.executable} -m pip install --force-reinstall --upgrade "plotly_express"

#
# Moving some of the guts to GitHub and refactoring 
#
!{sys.executable} -m pip install --upgrade --force-reinstall "git+https://github.com/HausReport/MightyLogic.git#egg=MightyLogic"

#
# Imports
#
from random import randrange
from scipy.signal import convolve2d
from enum import Enum, auto
from pprint import pprint
import plotly.figure_factory as ff

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import math

#import sklearn
#from sklearn.preprocessing import PowerTransformer, QuantileTransformer

  Cloning https://github.com/HausReport/MightyLogic.git to /tmp/pip-install-zkbsq3a4/mightylogic_dacc9bb4dcb94d39b37e223af6853c38
  Running command git clone -q https://github.com/HausReport/MightyLogic.git /tmp/pip-install-zkbsq3a4/mightylogic_dacc9bb4dcb94d39b37e223af6853c38
  Created wheel for MightyLogic: filename=MightyLogic-0.1-py3-none-any.whl size=7772 sha256=f941ac20b3af37316a18ffb49f19388949746e648eb056ccade419652e1a53b3
  Stored in directory: /tmp/pip-ephem-wheel-cache-k55g4n_p/wheels/84/7a/55/d74e49b550ce0fddb2379db726853920650df706dedea8daae
Successfully built MightyLogic


In [2]:

from MightyLogic.Currency import Currency as Currency
from MightyLogic.Values import Values as Values

Values.GOLD

0.6

# Turf War Demo

In [3]:
from MightyLogic.TurfWar.Tiles.BuildingFactory import *
from MightyLogic.TurfWar.TurfWarMap import TurfWarMap
#
# A Turf War Map
#

map2 =TurfWarMap()

# Use convenience functions to build value array
map2.addTile('A',1, mountains(3000,2000))
map2.addTile('A',2, forest(3000,0,2000))
map2.addTile('A',3, plains(3000, 2000))
map2.addTile('A',4, mountains(1600,1000))
map2.addTile('A',5, forest(3000,0,2000))
map2.addTile('A',6, plains(3000, 2000))

map2.addTile('B',1, factory(1650,500,5000))
map2.addTile('B',2, mountains(5000,5000))
map2.addTile('B',3, lighthouse(3500,300,7000))
map2.addTile('B',4, plains(5000,5000))
map2.addTile('B',5, mountains(5000,5000))
map2.addTile('B',6, gold_mine(90000,500,5000))

map2.addTile('C',1, forest(3000,0,2000))
map2.addTile('C',2, plains(5000,5000))
map2.addTile('C',3, mountains(5000,5000))
map2.addTile('C',4, forest(5000,0,5000))
map2.addTile('C',5, hells_gate(600,800,8500))
map2.addTile('C',6, mountains(3000,2000))

map2.addTile('D',1, mountains(3000,2000))
map2.addTile('D',2, witch_hut(3500,600, 7000))
map2.addTile('D',3, plains(5000,5000))
map2.addTile('D',4, altar(600,800,8500))
map2.addTile('D',5, forest(5000,0,5000))
map2.addTile('D',6, lighthouse(2000, 400, 5000))

map2.addTile('E',1, plains(3000, 2000))
map2.addTile('E',2, mountains(3000,2000))
map2.addTile('E',3, forest(3000,0,2000))
map2.addTile('E',4, plains(3000, 2000))
map2.addTile('E',5, mountains(3000,2000))
map2.addTile('E',6, forest(3000,0,2000))


# Payout array
image = map2.getValues()
pprint(image)

[[30003.0, 30003.0, 30003.0, 15001.6, 30003.0, 30003.0],
 [82582.5, 75005.0, 109535.0, 75005.0, 75005.0, 136500.0],
 [30003.0, 75005.0, 75005.0, 75005.0, 139560.0, 30003.0],
 [30003.0, 114035.0, 75005.0, 139560.0, 75005.0, 81020.0],
 [30003.0, 30003.0, 30003.0, 30003.0, 30003.0, 30003.0]]


In [4]:
# NSEW Averaging gives strategic value of a position
# I think of it as an image blur, but iirc it's equivalent to a discrete gradient operator
filter_kernel = [[0, .2, 0],
                 [.2, .2, .2],
                 [0, .2, 0]]

res = convolve2d(image, 
                 filter_kernel,
                 mode='same', 
                 boundary='fill', 
                 fillvalue=0)
pprint(res)

array([[ 28517.7 ,  33002.8 ,  36908.52,  30002.52,  30002.52,  39301.2 ],
       [ 43518.7 ,  74426.1 ,  72910.6 ,  69910.32,  91214.6 ,  54302.2 ],
       [ 43518.7 ,  73810.6 ,  81911.  , 100827.  ,  78915.6 ,  77416.6 ],
       [ 40808.8 ,  64810.2 ,  86721.6 ,  78915.6 ,  93029.6 ,  43206.2 ],
       [ 18001.8 ,  40808.8 ,  33002.8 ,  45913.8 ,  33002.8 ,  28205.2 ]])


In [5]:
#
# An approach that shows well-labeled heatmaps.
#

def show_heatmap(arr, title, round=True):
  flip = np.flip( np.array(arr), axis=0)  # use numpy to flip array vertically
  z = flip
  if round:
    z = z.round()

  x = ['1', '2', '3', '4','5','6']
  y = ['E','D','C','B','A']

  #z_text = ["%.0f" % number for number in z] - doesn't work

  z_text = flip.round(decimals=2)
  if round:
    z_text = flip.round( decimals=-1)

  # Possible to build up z_text to make it more friendly
  # z_text = [['Win', 'Lose', 'Win'],
  #          ['Lose', 'Lose', 'Win'],
  #          ['Win', 'Win', 'Lose']]

  fig = ff.create_annotated_heatmap(z, x=x, y=y, annotation_text=z_text, colorscale='Viridis')
  fig.update_layout(title=title, autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
  fig.show()



In [17]:
#
# SHOW BEST STAGING TILES
# =======================
# 1. If I'm a building, score is 0
# 2. Sum of neighbor buildings + my score
#

staging = map2.stagingScores() #'C',6)
show_heatmap(staging,"Staging Scores")

In [7]:
show_heatmap(image,"Payout Values for 3rd Rounds")
show_heatmap(res,"Strategic Values for Non-Payout Rounds")

In [8]:
fig = go.Figure(data=[go.Surface(z=image)])
fig.update_layout(title='Payout Value of Positions', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()

In [18]:
fig = go.Figure(data=[go.Surface(z=staging)])
fig.update_layout(title='Strategic Value of Positions', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()

In [10]:
payout = map2.getDataFrame()
payout

,1,2,3,4,5,6
A,30003.0,30003.0,30003.0,15001.6,30003.0,30003.0
B,82582.5,75005.0,109535.0,75005.0,75005.0,136500.0
C,30003.0,75005.0,75005.0,75005.0,139560.0,30003.0
D,30003.0,114035.0,75005.0,139560.0,75005.0,81020.0
E,30003.0,30003.0,30003.0,30003.0,30003.0,30003.0


In [11]:
# Puts data into super-clear dataframe, might use elsewhere.
oneTurn = map2.getStrategicDataFrame()
oneTurn

[[0.1560773647350679,
  -1.9869164565733701,
  0.04407185446571187,
  -1.7819651427781036,
  -1.224744871391589,
  0.7790113812382762],
 [0.1560773647350679,
  0.27558932252828106,
  0.04407185446571187,
  -0.4337485652867493,
  0.816496580927726,
  0.7790113812382762],
 [0.1560773647350679,
  0.5704423780150297,
  0.04407185446571187,
  0.7385712360216176,
  0.816496580927726,
  0.8894736067521375],
 [1.3178663821842396,
  0.5704423780150297,
  1.5127260501738127,
  0.7385712360216176,
  0.816496580927726,
  -1.223748184614345],
 [-1.7860984763894443,
  0.5704423780150297,
  -1.6449416135709483,
  0.7385712360216176,
  -1.224744871391589,
  -1.223748184614345]]

In [12]:
map2.grid_distance(0,0,2,2)

#grid_distance(0,0,2,2) == grid_distance(2,2,0,0)
#payout.iloc[row_dict['E']][6]

4

In [13]:
#
# Shows the maximum possible single tile you can take
# at a given turn.
#

payout = map2.getDataFrame()
show_heatmap(payout,"Strategic Positions for Turns 3n")

foo = map2.getStrategicDataFrame()
show_heatmap(foo,"Strategic Positions for Turns 3n+2")

#thisThing = pd.DataFrame(foo, index=["A","B","C","D","E"], columns=[1,2,3,4,5,6])
#bar = oneMove(thisThing)
#show_heatmap(bar,"Strategic Positions for Turns 3n+1")




In [14]:
#
# Shows payout shares for holding a tile on round 3n 
#

gm = map2.getTile('C',5) #b6 # c5 # gold_mine(90000,500,5000)
print(gm.payouts())


Payouts for C:5 Hell's Gate
Place               Reward
First               42 epic souls, 595 contribution
2nd                 36 epic souls, 510 contribution
3rd                 30 epic souls, 425 contribution
4-6                 24 epic souls, 340 contribution
7-11                18 epic souls, 255 contribution
12-25               12 epic souls, 170 contribution
26-44               6 epic souls, 85 contribution



In [15]:
pprint(map2.arr)

{('A', 1): Tile(name=Mountains, row=A, column=1),
 ('A', 2): Tile(name=Forest, row=A, column=2),
 ('A', 3): Tile(name=Plains, row=A, column=3),
 ('A', 4): Tile(name=Mountains, row=A, column=4),
 ('A', 5): Tile(name=Forest, row=A, column=5),
 ('A', 6): Tile(name=Plains, row=A, column=6),
 ('B', 1): Tile(name=Factory, row=B, column=1),
 ('B', 2): Tile(name=Mountains, row=B, column=2),
 ('B', 3): Tile(name=Lighthouse, row=B, column=3),
 ('B', 4): Tile(name=Plains, row=B, column=4),
 ('B', 5): Tile(name=Mountains, row=B, column=5),
 ('B', 6): Tile(name=Gold Mine, row=B, column=6),
 ('C', 1): Tile(name=Forest, row=C, column=1),
 ('C', 2): Tile(name=Plains, row=C, column=2),
 ('C', 3): Tile(name=Mountains, row=C, column=3),
 ('C', 4): Tile(name=Forest, row=C, column=4),
 ('C', 5): Tile(name=Hell's Gate, row=C, column=5),
 ('C', 6): Tile(name=Mountains, row=C, column=6),
 ('D', 1): Tile(name=Mountains, row=D, column=1),
 ('D', 2): Tile(name=Witch Hut, row=D, column=2),
 ('D', 3): Tile(name=Pl

In [16]:
map2.buildingList()
print(map2.buildingListText())

Payouts for C:5 Hell's Gate
Place               Reward
First               42 epic souls, 595 contribution
2nd                 36 epic souls, 510 contribution
3rd                 30 epic souls, 425 contribution
4-6                 24 epic souls, 340 contribution
7-11                18 epic souls, 255 contribution
12-25               12 epic souls, 170 contribution
26-44               6 epic souls, 85 contribution

Payouts for D:4 Altar
Place               Reward
First               42 epic souls, 595 contribution
2nd                 36 epic souls, 510 contribution
3rd                 30 epic souls, 425 contribution
4-6                 24 epic souls, 340 contribution
7-11                18 epic souls, 255 contribution
12-25               12 epic souls, 170 contribution
26-44               6 epic souls, 85 contribution

Payouts for B:6 Gold Mine
Place               Reward
First               6,300 gold, 350 contribution
2nd                 5,400 gold, 300 contribution
3rd                

# Notes


1.   Annotated heatmap: https://plotly.com/python/annotated-heatmap/
2.   A-Star search on grid in python at https://github.com/BaijayantaRoy/Medium-Article/blob/master/A_Star.ipynb

